# Homework 2 Part 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
%matplotlib inline

#### Q1 (2 points)

a. Implement a function that takes in the *train.csv* dataset and returns a cleaned dataset without outliers and NAs.

In [2]:
train = pd.read_csv('train.csv', index_col = 0)

def process(train):
    ## Your code goes here
    train = train.dropna()
    processed_train = train[(abs(stats.zscore(train)) < 3).all(axis=1)]
    return processed_train

b. Implement a function that takes in the R<sup>2</sup> of a regression and return the VIF value. 

In [3]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1 - r_squared)
    return vif

c. Implement a function that takes in preprocessed data and returns a dataframe that contains the VIF for each variable. The dataframe should contain two columns: variable names and their VIFs.

In [4]:
processed_train = process(train) # Getting the cleaned data

def generate_vif_dataframe(processed_train):
    ## Your code goes here
    vif_scores = []
    var = []
    
    new_data = processed_train.drop(['quality'], axis = 1)
    
    for i in new_data.columns:
        
        var = [i]
        x = new_data.drop(i,axis=1).values
        y = new_data[i].values
        
        #calculate regression
        s = sm.add_constant(x)
        results = sm.OLS(y,s).fit()
        r2 = results.rsquared
        
        vif = round(calculate_vif(r2),3)
        var.append(vif)
        vif_scores.append(var)
        
        vif_dataframe = pd.DataFrame(vif_scores, columns = ['Variable', 'VIF'])   
    
    return vif_dataframe

d. Use the functions that you developed from Q1 and generate a VIF dataframe for all the variables in the wine dataset. Discuss the VIF values you find and the effect of multicollinearity on regression models.

In [5]:
## Your code goes here
vif = generate_vif_dataframe(processed_train)
vif

Variable    VIF
0          fixed acidity  7.973
1       volatile acidity  1.823
2            citric acid  3.092
3         residual sugar  1.651
4              chlorides  1.203
5    free sulfur dioxide  2.025
6   total sulfur dioxide  2.254
7                density  6.432
8                     pH  3.285
9              sulphates  1.226
10               alcohol  3.146

In [6]:
## Your discussion goes here

# According to my results, 'alcohol' is the largest VIF, which implies any changes to this variable could result
# in a larger variance in the outputs compared to the other explanatory variables.

#### Q2 (4 points)

a. Use the VIF dataframe you generated from Q1 and choose a subset of variables that does not have high multicollinearity.

In [137]:
## Your code goes here

subset = processed_train[['residual sugar','chlorides','total sulfur dioxide']]
#subset

b. Fit a multiple linear regression model and print a summary of the regression result.

In [136]:
## Your code goes here
x = subset
y = processed_train['quality']

s = sm.add_constant(x)
prediction = sm.OLS(y,s).fit()
#print(prediction.summary())

c. Use 5-fold cross validation to validate the mean squared error (MSE) of a multiple linear regression model fit to the subset of data. (HINT: You will need to implement a function to calculate the MSE. See this link for more details: https://en.wikipedia.org/wiki/Mean_squared_error).

In [60]:
## Your code goes here

# kfold function will split the data "x" into "k" equal-sized data sets
def kfold(x,k):
    i = 0 
    while i < k:
        xtrain = np.split(x,k)
        return xtrain
        i += 1

# linreg function performs a multi-linear regression thru the x data sets & return the coefficients
def linreg(x,y):
    s = sm.add_constant(x)
    r = sm.OLS(y,s).fit()
    return r.params

# Using all the data except for the last set as my training set

# Calculating the parameters for y estimations
# Order of parameters: constant, sugar, chlorides, free sulfur dioxide


# Calculating yhat = constant + b1*x1 + b2*x2 + b3*x3

def yhat(parameters,x):
    yhat = parameters[0] + parameters[1]*x.iloc[:,0] + parameters[2]*x.iloc[:,1] + parameters[3]*x.iloc[:,2]
    #+ parameters[4]*x.iloc[:,3] + parameters[5]*x.iloc[:,4] + parameters[6]*x.iloc[:,5] + parameters[7]*x.iloc[:,6] 
    #+ parameters[8]*x.iloc[:,7] + parameters[9]*x.iloc[:,8]
    return yhat


# MSE = (1/n) * sum[(yi - yhat)^2]

def mse(yi,yhat):
    n = len(yi)
    mse_score = (1/n) * np.sum((yi - yhat)**2)
    return mse_score

# Calculating the MSE between the actual quality given & the estimated quality from multi-linear regression

In [61]:
x1,x2,x3,x4,x5 = kfold(x,5)
y1,y2,y3,y4,y5 = kfold(y,5)

xtrain1 = pd.concat([x1,x2,x3,x4])
ytrain1 = pd.concat([y1,y2,y3,y4])
xtest1 = x5
ytest1 = y5

xtrain2 = pd.concat([x2,x3,x4,x5])
ytrain2 = pd.concat([y2,y3,y4,y5])
xtest2 = x1
ytest2 = y1

xtrain3 = pd.concat([x1,x3,x4,x5])
ytrain3 = pd.concat([y1,y3,y4,y5])
xtest3 = x2
ytest3 = y2

xtrain4 = pd.concat([x1,x2,x4,x5])
ytrain4 = pd.concat([y1,y2,y4,y5])
xtest4 = x3
ytest4 = y3

xtrain5 = pd.concat([x1,x2,x3,x5])
ytrain5 = pd.concat([y1,y2,y3,y5])
xtest5 = x4
ytest5 = y4

In [66]:
def main(xtrain,ytrain,xtest,ytest):
    coeff = linreg(xtrain,ytrain)
    y_hat = yhat(coeff,xtest)
    mse_val = mse(y_hat,ytest)
    return mse_val

def avg_mse(mse1,mse2,mse3,mse4,mse5):
    final_mse = round((mse1 + mse2 + mse3 + mse4 + mse5)/5,4)
    print("The average MSE is",final_mse)

In [67]:
mse1 = main(xtrain1,ytrain1,xtest1,ytest1)
mse2 = main(xtrain2,ytrain2,xtest2,ytest2)
mse3 = main(xtrain3,ytrain3,xtest3,ytest3)
mse4 = main(xtrain4,ytrain4,xtest4,ytest4)
mse5 = main(xtrain5,ytrain5,xtest5,ytest5)

avg_mse(mse1,mse2,mse3,mse4,mse5)

The average MSE is 0.6185


d. Make a prediction on the test set using the model you fit in 2b and submit to Kaggle.

In [139]:
## Your code goes here

test = pd.read_csv('test.csv', index_col = 0)

x2test = test[['fixed acidity','density']]

x2train = processed_train[['fixed acidity','density']]
quality = processed_train['quality']

new_para = linreg(x2train,quality)
pred = new_para[0] + new_para[1]*x2test.iloc[:,0] + new_para[2]*x2test.iloc[:,1]  #+ new_para[3]*x2test.iloc[:,2] + new_para[4]*x2test.iloc[:,3]  + new_para[5]*x2test.iloc[:,4]

test['predicted'] = pred
#test.to_csv(r"C:\Users\Duker\Desktop\CS530\Homework\HW2\attempt4.csv")

e. Compare the Kaggle score and the cross-validation result. Is the cross-validation result a good representation of the prediction accuracy on the test data?

In [15]:
# Cross validation result from test data set:
# First Kaggle score is 0.6429
# First MSE score is 0.6185

In [16]:
## Your discussion here

# My MSE score on the training data set is relatively close to the Kaggle score so I think the cross-validation result
# is a good representation of the accuracy. 

f. Discuss the difference between goodness of fit on the training set, on the validation set, and on the test set.

In [17]:
## Your discussion here

# According to the MSE's from each set, the test set seems to fit a lot better than the training & c-v set.
# This is simply could be because the test data contains much less data points compared to the other 2.
# However, with the cross-validation method & selecting necessary variables with low VIF scores, I was able to avoid the 
# issues of under-fitting & over-fitting.